In [ ]:
library(heatmaply)
library(scFunctions)
library(tidyverse)
#library(SCENIC)
#library(SeuratDisk)
library(pheatmap)
library(RColorBrewer)
library(dendextend)
library(reshape2)
source("/home/jovyan/zxli_SCI/script/scenic/CSI.R")

In [71]:
save_pheatmap_pdf <- function(x, filename, width=10, height=10) {
   stopifnot(!missing(x))
   stopifnot(!missing(filename))
   pdf(filename, width=width, height=height)
   grid::grid.newpage()
   grid::grid.draw(x$gtable)
   dev.off()
}

### regulon genes scale expression

In [73]:
da<-readRDS("/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/res02_220310/WT.merge.replace_v2.SCT.regress_CC.nC.mt.ident.pc20.k50.res02.rds")

In [ ]:
regulongenes<-gsub("...$","",colnames(auc_mtx))
regulongenes<-intersect(regulongenes,rownames(da))
length(regulongenes)
head(regulongenes)

In [ ]:
ma<-da@assays$SCT@scale.data[regulongenes,]
ma<-as.matrix(ma)
dim(ma)

In [84]:
write.csv(ma,"regulongenes.SCT.scale.data.csv")

In [ ]:
ma2<-t(ma)
head(ma2)

In [ ]:
c1<-cor(ma2)
c1[1:5,1:5]

In [87]:
write.csv(c1,"regulongenes.SCT.scale.data.pearson.correlation.csv")

In [88]:
pcc<-as.data.frame(c1)
pcc$regulon_1 <- rownames(pcc)
  pcc <- pcc %>%
    gather(regulon_2,pcc,-regulon_1) %>%
    mutate("regulon_pair" = paste(regulon_1,regulon_2,sep="_"))

### SCT auc data as input : regulon auc_mtx binarized by pyscenic

In [ ]:
wd<-"/home/jovyan/zxli_SCI/result/pyscenic/WT.merge.replace_v2/CSI/pc20k50res02/time_domain/"
if(!dir.exists(wd))
    dir.create(wd)
setwd(wd)
getwd()

In [ ]:
auc_mtx<-read.csv("/home/jovyan/zxli_SCI/result/pyscenic/WT.merge.replace_v2/SCT/auc_mtx.csv")
rownames(auc_mtx)<-auc_mtx[,1]
auc_mtx<-auc_mtx[,-1]
head(auc_mtx)

In [ ]:
binary_auc<-read.csv("../../WT.SCT.binarizedbyPyscenic.auc_mtx.csv")
rownames(binary_auc)<-binary_auc$Cell
binary_auc<-binary_auc[,-1]
head(binary_auc)

In [ ]:
#transposition the matrix
binary_regulons_trans<-as.matrix(t(binary_auc))
binary_regulons_trans[1:4,1:4]

In [ ]:
regulons_trans<-as.matrix(t(auc_mtx))
regulons_trans[1:4,1:4]

### calculate regulon specificty score (RSS)

In [ ]:
#da <- readRDS("/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/SCT/WT.merge.replace_v2.SCT.regress_CC.nC.mt.ident.pc20.k50.res025.rds")
#meta<-da@meta.data
meta<-read.csv("/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/res02_220310/WT.SCT.pc20.k50.res02.meta.data.csv")
rownames(meta)<-meta[,1]
meta<-meta[,-1]
head(meta)

In [ ]:
meta$time_domain<-paste0(meta$time,"_",meta$domain_res02)
unique(meta$time_domain)
#unique(meta$SCT_snn_res.0.2)
#head(meta)

In [ ]:
#meta$cell_type<-paste0("cluster_",meta$SCT_snn_res.0.2)
#meta$cell_type<-paste0(meta$time,"_",meta$domain)
meta$cell_type<-meta$time_domain
unique(meta$cell_type)

In [ ]:
# calculate the RSS for all regulons over all spatial clusters
rrs_df<-calculate_rrs(meta,binary_regulons = binary_regulons_trans,cell_type_column = "cell_type")
#rrs_df2<-calculate_rrs(meta2,binary_regulons = bi_reg_trans,cell_type_column = "cell_type2")
#rrs_df<-calcRSS(AUC = regulons_trans,cellAnnotation = meta$cell_type)
#rrs_df[1:5,1:5]

In [ ]:
library(reshape2)
#rn<-rep(rownames(rrs_df),ncol(rrs_df))
rss<-melt(rrs_df)
#rss$regulon<-rn
colnames(rss)<-c("regulon","cell_type","RSS")
head(rss)

In [69]:
write.csv(rss,"WT.regulons.res02_time-domain.rss.dataframe.csv")

In [287]:
write.csv(rrs_df,"WT.binary.regulons.res02_time-domain.rrs.dataframe.csv")


In [ ]:
rrs_df<-read.csv("WT.binary.regulons.res02_time-domain.rrs.dataframe.csv")
rownames(rrs_df)<-rrs_df[,1]
rrs_df<-rrs_df[,-1]
head(rrs_df)

In [ ]:
rrs_df %>% 
    group_by(cell_type) %>%
    top_n(.,n=20,wt=RSS) ->rss_sub
dim(rss_sub)

In [ ]:
#subR<-unique(rss_sub$regulon)
subR<-unique(rss_sub$regulon)
length(subR)

In [ ]:
# visualize all regulons over all cell types using heatmaps. Let's first investigate the distribution of RSS over all cell types
library(ggridges)
range(rrs_df$RSS)

In [ ]:
rrs_df_wide<-rrs_df %>% spread(cell_type,RSS)
rownames(rrs_df_wide)<-rrs_df_wide$regulon
rrs_df_wide<-rrs_df_wide[,2:ncol(rrs_df_wide)]
head(rrs_df_wide)

In [ ]:
## subset all regulons that don't have at least an RSS of 0.2 for one cell type
rrs_df_wide_specific<-rrs_df_wide[subR,]#[apply(rrs_df_wide,MARGIN = 1,FUN = function(x) any(x>0.25)),]
#rrs_df_wide_specific<-rrs_df_wide[subR,]
dim(rrs_df_wide_specific)
                                        

In [245]:
# order col
rrs_df_wide_specific<-rrs_df_wide_specific[,#c('WT_sham_DH_0.5mm','WT_3h_DH_0.5mm','WT_24h_DH_0.5mm',
                                             # 'WT_sham_DH_1mm','WT_3h_DH_1mm','WT_24h_DH_1mm','WT_72h_DH_1mm',
                                             # 'WT_sham_MG_0.5mm','WT_3h_MG_0.5mm','WT_24h_MG_0.5mm','WT_72h_MG_0.5mm',
                                             # 'WT_sham_MG_1mm','WT_3h_MG_1mm','WT_24h_MG_1mm','WT_72h_MG_1mm',
                                             # 'WT_sham_VH_0.5mm','WT_3h_VH_0.5mm','WT_24h_VH_0.5mm','WT_72h_VH_0.5mm',
                                             # 'WT_sham_VH_1mm','WT_3h_VH_1mm','WT_24h_VH_1mm','WT_72h_VH_1mm',
                                             # 'WT_sham_WM_0.5mm','WT_3h_WM_0.5mm','WT_24h_WM_0.5mm','WT_72h_WM_0.5mm',
                                             # 'WT_sham_WM_1mm','WT_3h_WM_1mm','WT_24h_WM_1mm','WT_72h_WM_1mm')
                                           c('WT_sham_DH','WT_3h_DH','WT_24h_DH','WT_72h_DH',
                                              'WT_sham_MG','WT_3h_MG','WT_24h_MG','WT_72h_MG',
                                              'WT_sham_VH','WT_3h_VH','WT_24h_VH','WT_72h_VH',
                                              'WT_sham_WM','WT_3h_WM','WT_24h_WM','WT_72h_WM')
                                           #c("cluster_0","cluster_1","cluster_2","cluster_3",
                                           # "cluster_4","cluster_5","cluster_6","cluster_7")
                                          ]

In [13]:
# visulaize the regulons that show an RSS over the defined threshold using heatmapply, a heatmap library using plotly.
library(heatmaply)
library(plotly)
library(htmlwidgets)

In [ ]:
head(rrs_df_wide_specific)
col_anno<-data.frame(row.names = colnames(rrs_df_wide_specific),"domain"=c(rep("WM",4),rep("MG",4),rep("DH",4),rep("VH",4)),
                    "time"=c(rep(c("sham","3h","24h","72h"),4)))
col_anno

In [55]:
domain_col<-c('#20854EA8','#0072B5A8','#BC3C29A8','#E18727A8')
names(domain_col)<-c("WM", "MG", "DH", "VH")
time_col<-c('#374E55FF','#DF8F44FF','#00A1D5FF','#B24745FF')
names(time_col)<-c("sham","3h","24h","72h")

In [ ]:
col_list<-list(domain=domain_col,time=time_col)
col_list

In [57]:
gap_col<-c(4,8,12)

In [ ]:
d_hc<-hclust(dist(rrs_df_wide_specific))
den<-as.dendrogram(hclust(dist(rrs_df_wide_specific)))
#den<-rotate(den,49:1)
#den<-color_branches(den,k=7)
##labels_colors(den)<-rainbow_hcl(7)[sort_levels_values(
##                    as.numeric(rownames()))]
#den<-hang.dendrogram(den,hang_height = 0.1)
#plot(den,horiz=T)

In [ ]:
options(repr.plot.width=6,repr.plot.height=7)
#pdf("WT.regulon.RSS.thrs01.cluster.heatmaply.pdf",width=8,height=8)
p<-pheatmap(rrs_df_wide_specific,col=colorRampPalette(rev(brewer.pal(10,"Spectral")))(50),scale="none",
          cluster_cols=F,cluster_rows=T,treeheight_row=10,cutree_rows=7,annotation_col=col_anno,border_color="NA",
         annotation_colors=col_list,legend=T,gaps_col=gap_col,
            show_rownames=T#,legend_labels=c("RSS")
          #file = "WT.SCT.pct01.data.regulons.01wide.RSS.of.clusters.pheatmap.html",
          ) 

In [ ]:
save_pheatmap_pdf(p,"WT.res02.time-domain.binary.regulon.RSS.top3.pheatmap.pdf",width = 6,height = 7)

In [ ]:
options(repr.plot.width=6,repr.plot.height=10)
#pdf("WT.regulon.RSS.thrs01.cluster.heatmaply.pdf",width=8,height=8)
p<-pheatmap(rrs_df_wide_specific,col=colorRampPalette(rev(brewer.pal(10,"Spectral")))(50),scale="none",
          cluster_cols=F,cluster_rows=T,treeheight_row=10,cutree_rows=7,annotation_col=col_anno,border_color="NA",
         annotation_colors=col_list,legend=T,gaps_col=gap_col,
            show_rownames=F#,legend_labels=c("RSS")
          #file = "WT.SCT.pct01.data.regulons.01wide.RSS.of.clusters.pheatmap.html",
          ) 

### calculate connection specificity index (CSI) for all regulons/ selected top regulons
- the final statistics that we want to calculate is the connection specificity index. The CSI is a major connectedness between the different regulons. Regulons that share high CSI likely are co-regulating downstream genes and are together responsible for cell function

In [ ]:
#regulonAUC<-t(auc_mtx)
regulonAUC<-t(binary_auc)
head(regulonAUC)

In [ ]:
regulonAUC_sub<-regulonAUC[subR,]
dim(regulonAUC_sub)

In [143]:
regulons_csi <- calculate_csi(regulonAUC_sub,
                              calc_extended = FALSE)

In [ ]:
#test<-binary_auc[,subR]
test_sub<-t(test)
head(test)

In [63]:
pearson_cor <- cor(test)
  pearson_cor_df <- as.data.frame(pearson_cor)
  pearson_cor_df$regulon_1 <- rownames(pearson_cor_df)
  pearson_cor_long <- pearson_cor_df %>%
    gather(regulon_2,pcc,-regulon_1) %>%
    mutate("regulon_pair" = paste(regulon_1,regulon_2,sep="_"))

In [ ]:
#head(pcc)
#pcc$edge<-abs(pcc$pcc)
pcc$PN<-NA
pcc[pcc$pcc<0,"PN"]<-"0"
pcc[pcc$pcc>0,"PN"]<-"1"
pcc$PN<-as.factor(pcc$PN)
head(pcc)

In [ ]:
pearson_cor_long<-merge(pcc,regulons_csi[,c("regulon_pair","CSI")],by = "regulon_pair")
head(pearson_cor_long)

In [ ]:
write.csv(pearson_cor_long,"time_domain.RSS.top20.regulons.genescaleexp_correlation.and.csi.df.csv")
dim(pearson_cor_long)

In [ ]:
pearson_cor_long<-read.csv("time_domain.RSS.top20.regulons.correlation.and.csi.df.csv")
head(pearson_cor_long)

In [ ]:
pearson_cor_long<-pearson_cor_long[abs(pearson_cor_long$pcc)>0.65#&pearson_cor_long$CSI>0
                                   ,]
pearson_cor_long<-pearson_cor_long[!pearson_cor_long$regulon_1==pearson_cor_long$regulon_2,]
dim(pearson_cor_long)

In [ ]:
pcc_df_filt<-pearson_cor_long#[!pearson_cor_long$regulon_pair %in% rm_regulon_c,]
table(pcc_df_filt$PN)

In [ ]:
cluster_df_sub<-clusters_df[clusters_df$regulon %in% unique(c(pcc_df_filt$regulon_1,pcc_df_filt$regulon_2)),]
table(cluster_df_sub$csi_cluster)

In [ ]:
reg<-unique(c(pcc_df_filt$regulon_1,pcc_df_filt$regulon_2))
length(reg)

In [218]:
write.csv(pcc_df_filt,"rss.top20.auc.regulon.auc_pearson.correlation065.long.CSI.csv",quote=FALSE)

In [219]:
write.csv(cluster_df_sub,"rss.top20.auc.regulon.auc_pearson.correlation065.long.CSI.nodes.csv",quote=FALSE)

In [171]:
write.csv(pearson_cor_long,"rss.top20.regulon.pearson.correlation.long.csv",quote=FALSE)

In [102]:
write.csv(regulons_csi,"WT.SCT.binary.regulons.SCT.res02.time-domain.RSS.top20.csi.csv")

In [ ]:
regulons_csi<-read.csv("WT.SCT.binary.regulons.SCT.res02.time-domain.RSS.top20.csi.csv")
rownames(regulons_csi)<-regulons_csi[,1]
regulons_csi<-regulons_csi[,-1]
regulons_csi$regulon_1<-gsub("...$","",regulons_csi$regulon_1)
regulons_csi$regulon_2<-gsub("...$","",regulons_csi$regulon_2)
head(regulons_csi)
regulons_csi$regulon_pair<-paste0(regulons_csi$regulon_1,"_",regulons_csi$regulon_2)
head(regulons_csi)

In [ ]:
# finnally, we can also export the SCI cluster modules by the following code with the same cluster number we used on to create the heatmap
csi_csi_wide<- regulons_csi %>% 
    spread(regulon_2,CSI)

future_rownames<-csi_csi_wide$regulon_1
csi_csi_wide<-as.matrix(csi_csi_wide[,2:ncol(csi_csi_wide)])
rownames(csi_csi_wide)<-future_rownames
csi_csi_wide[1:5,1:5]

In [154]:
write.csv(csi_csi_wide,"WT.SCT.res02.time-domain.RSS_top20.binary_csi.wide.matrix.csv")

In [ ]:
csi_csi_wide<-read.csv("WT.SCT.res02.time-domain.RSS_top20.binary_csi.wide.matrix.csv")
rownames(csi_csi_wide)<-csi_csi_wide[,1]
csi_csi_wide<-csi_csi_wide[,-1]
head(csi_csi_wide)

In [ ]:
subR<-rownames(csi_csi_wide)
length(subR)

In [ ]:
options(repr.plot.width=12,repr.plot.height=12)
p<- pheatmap(csi_csi_wide,
           show_colnames = FALSE,
           show_rownames=FALSE,
           display_numbers=F,
           color = viridis(n = 25),
           cutree_cols = 7,
           cutree_rows = 7,
           #fontsize_row = 0,#font_size_regulons,
           cluster_cols = TRUE,
           cluster_rows = TRUE,
           treeheight_row = 5,
           treeheight_col = 5,
           clustering_method="complete",
           clustering_distance_rows = "euclidean",
           clustering_distance_cols = "euclidean",
           width = 300,
           height = 300)
  #dev.off()

In [47]:
regulons_hclust<-hclust(dist(csi_csi_wide,method="euclidean"),method = "complete")

In [48]:
clusters<-cutree(regulons_hclust,k=7)
clusters_df<-data.frame("regulon"=names(clusters),
                       "csi_cluster"=clusters)

In [ ]:
anno_row<-data.frame(row.names = rownames(clusters_df),"CSI_module"=as.character(clusters_df[,2]))

head(anno_row)

In [ ]:
anno_col<-brewer.pal(7,"Set3")
names(anno_col)<-unique(anno_row$CSI_module)
anno_col<-list(CSI_module=anno_col)
anno_col

In [ ]:
p<- pheatmap(csi_csi_wide,annotation_row = anno_row,annotation_colors = anno_col,
           show_colnames = FALSE,
           show_rownames=FALSE,
           display_numbers=F,
           color = viridis(n = 25),
           cutree_cols = 7,
           cutree_rows = 7,
           #fontsize_row = 0,#font_size_regulons,
           cluster_cols = TRUE,
           cluster_rows = TRUE,
           treeheight_row = 5,
           treeheight_col = 5,
           clustering_method="complete",
           clustering_distance_ro0ws = "euclidean",
           clustering_distance_cols = "euclidean",
           width = 300,
           height = 800)
  #dev.off()

In [ ]:
save_pheatmap_pdf(p,"WT.res02_time_domain.rss_top20.csi.complete_nclust7.hcluster.withAnno.pheatmap.pdf")

In [ ]:
## export module node dataframe to cytoscape for regulatory network visualization
head(clusters_df)
rownames(clusters_df)<-gsub("...$","",rownames(clusters_df))
clusters_df$regulon<-gsub("...$","",clusters_df$regulon)
head(clusters_df)
write.csv(clusters_df,"rss.top20.regulons.csi.7hcluster.df.csv",quote=FALSE)

In [ ]:
clusters_df<-read.csv("rss.top20.regulons.csi.7hcluster.df.csv")
head(clusters_df)

In [ ]:
regulons <- unique(clusters_df$regulon)
length(regulons)

In [30]:
rownames(regulonAUC_sub)<-gsub("...$","",rownames(regulonAUC_sub))

In [31]:
## Finally, let's calculate activity scores for each csi module based on the specifity scores of all the regulons in that module.
# For this, we will load the AUC matrix again and calculate the mean of AUC fr each regulon per cell type and add up these scores per module
csi_cluster_activity_wide<-calc_csi_module_activity(clusters_df,regulonAUC_sub,metadata = meta,cell_type_column = "cell_type")

In [ ]:
#csi_cluster_activity_wide<-csi_cluster_activity_wide[,c('WT_sham_DH_0.5mm','WT_3h_DH_0.5mm','WT_24h_DH_0.5mm',
#                                              'WT_sham_DH_1mm','WT_3h_DH_1mm','WT_24h_DH_1mm','WT_72h_DH_1mm',
#                                              'WT_sham_MG_0.5mm','WT_3h_MG_0.5mm','WT_24h_MG_0.5mm','WT_72h_MG_0.5mm',
#                                              'WT_sham_MG_1mm','WT_3h_MG_1mm','WT_24h_MG_1mm','WT_72h_MG_1mm',
#                                              'WT_sham_VH_0.5mm','WT_3h_VH_0.5mm','WT_24h_VH_0.5mm','WT_72h_VH_0.5mm',
#                                              'WT_sham_VH_1mm','WT_3h_VH_1mm','WT_24h_VH_1mm','WT_72h_VH_1mm',
#                                              'WT_sham_WM_0.5mm','WT_3h_WM_0.5mm','WT_24h_WM_0.5mm','WT_72h_WM_0.5mm',
#                                              'WT_sham_WM_1mm','WT_3h_WM_1mm','WT_24h_WM_1mm','WT_72h_WM_1mm')]
csi_cluster_activity_wide<-csi_cluster_activity_wide[,
                                                     c('WT_sham_WM','WT_3h_WM','WT_24h_WM','WT_72h_WM',
                                                       'WT_sham_MG','WT_3h_MG','WT_24h_MG','WT_72h_MG',
                                                       'WT_sham_DH','WT_3h_DH','WT_24h_DH',"WT_72h_DH",
                                              'WT_sham_VH','WT_3h_VH','WT_24h_VH','WT_72h_VH'
                                              
                                              
                                              #       c("cluster_0","cluster_1","cluster_2","cluster_3","cluster_4","cluster_5",
                                              #        "cluster_6"#,"cluster_7"
                                              )]
head(csi_cluster_activity_wide)

In [ ]:
csi_long<-melt(t(scale(t(csi_cluster_activity_wide),center = T,scale = T)),value.name = "csi",varnames = "cluster_csi")#,variable.name = "cluster_exp"
colnames(csi_long)<-c("cluster_csi","cluster_exp","csi")
head(csi_long)

In [62]:
top3<-csi_long %>% group_by(cluster_exp) %>% top_n(.,wt = csi,n = 1)

In [63]:
top3$cluster_csi = factor(top3$cluster_csi,levels = rev(unique(top3$cluster_csi)))

In [64]:
csi_cluster_wide<-csi_cluster_activity_wide[c(2,3,1,6,4,5,7),#unique(as.character(top3$cluster_csi)),
                                           ]

In [ ]:
col_anno<-data.frame(row.names = colnames(csi_cluster_wide),"domain"=c(rep("WM",4),rep("MG",4),rep("DH",4),rep("VH",4)),
                    "time"=c(rep(c("sham","3h","24h","72h"),4)))
col_anno

In [ ]:
anno_row<-data.frame(row.names = unique(clusters_df$csi_cluster),"CSI_module"=as.character(unique(clusters_df$csi_cluster)))
head(anno_row)
anno_col<-brewer.pal(7,"Set3")
names(anno_col)<-anno_row$CSI_module
col_list<-list(CSI_module=anno_col,domain=domain_col,time=time_col)
col_list

In [ ]:
options(repr.plot.width=6,repr.plot.height=5)
p<-pheatmap(csi_cluster_wide,border_color = "NA",
           show_colnames = TRUE,
           color = viridis(n = 100),#colorRampPalette(rev(brewer.pal(6,"RdYlBu")))(50),#viridis(n = 100),
           cluster_cols = F,
           cluster_rows = F,
           annotation_col=col_anno,
           annotation_colors=col_list,
            legend=T,
            gaps_col=gap_col,
            annotation_row=anno_row,
            
           #scale="row",
        
           #clustering_distance_rows = "euclidean"#,
           #clustering_distance_cols = "euclidean"
            )
save_pheatmap_pdf(p,"WT.SCT.binary.regulon.res02_time-domain.rss_top20.7csi.heatmap.viridis_220720.pdf")

#### 7 csi module activity in WM clusters

In [ ]:
meta<-read.csv("/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/res02_220310/WT.SCT.pc20.k50.res02.meta.data.csv")
rownames(meta)<-meta[,1]
meta<-meta[,-1]
head(meta)

In [ ]:
binary_auc<-read.csv("../../WT.SCT.binarizedbyPyscenic.auc_mtx.csv")
rownames(binary_auc)<-binary_auc$Cell
binary_auc<-binary_auc[,-1]
colnames(binary_auc)<-gsub("...$","",colnames(binary_auc))
head(binary_auc)

In [ ]:
regulonAUC<-t(binary_auc)
head(regulonAUC)

In [ ]:
clusters_df<-read.csv("rss.top20.regulons.csi.7hcluster.df.csv")
head(clusters_df)

In [ ]:
regulons <- unique(clusters_df$regulon)
length(regulons)

In [ ]:
meta_s<-meta[meta$domain_res02=="WM",]
dim(meta_s)

In [ ]:
meta_s$time_cluster<-paste0(meta_s$time,"-",meta_s$SCT_snn_res.0.2)
table(meta_s$time_cluster)
meta_s<-meta_s[!meta_s$time_cluster=="WT_sham-3",]
table(meta_s$time_cluster)

In [ ]:
regulonAUC_s<-regulonAUC[,rownames(meta_s)]
dim(regulonAUC_s)

In [23]:
csi_cluster_activity_wide<-calc_csi_module_activity(clusters_df,regulonAUC_s,metadata = meta_s,cell_type_column = "time_cluster")

In [ ]:
csi_cluster_activity_wide<-csi_cluster_activity_wide[,
                                                     c('WT_sham-0','WT_3h-0','WT_24h-0','WT_72h-0',
                                                       'WT_sham-4','WT_3h-4','WT_24h-4','WT_72h-4',
                                                       'WT_24h-3','WT_72h-3'
                                              #       c("cluster_0","cluster_1","cluster_2","cluster_3","cluster_4","cluster_5",
                                              #        "cluster_6"#,"cluster_7"
                                              )]
head(csi_cluster_activity_wide)

In [31]:
csi_cluster_wide<-csi_cluster_activity_wide[c(4,6,7,2,3,5,1),#unique(as.character(top3$cluster_csi)),
                                           ]

In [ ]:
cluster_col<-c('#66AB94','#E2E293','#C580AA')
names(cluster_col)<-c("0", "3", "4")
time_col<-c('#374E55FF','#DF8F44FF','#00A1D5FF','#B24745FF')
names(time_col)<-c("sham","3h","24h","72h")
#gap_col<-c(2,4,7)
gap_col<-c(4,8)
anno_row<-data.frame(row.names = unique(clusters_df$csi_cluster),"CSI_module"=as.character(unique(clusters_df$csi_cluster)))
head(anno_row)
anno_col<-brewer.pal(7,"Set3")
names(anno_col)<-anno_row$CSI_module
col_list<-list(CSI_module=anno_col,cluster=cluster_col,time=time_col)
col_list
col_anno<-data.frame(row.names = colnames(csi_cluster_wide),
                     #"cluster"=c(rep(c("0","4"),2),rep(c("0","3","4"),2)),
                     "cluster"=c(rep("0",4),rep("4",4),rep("3",2)),
                    #"time"=c(rep("sham",2),rep("3h",2),rep("24h",3),rep("72h",3))
                     "time"=c(rep(c("sham","3h","24h","72h"),2),"24h","72h")
                    )
col_anno

In [ ]:
options(repr.plot.width=6,repr.plot.height=5)
p<-pheatmap(csi_cluster_wide,border_color = "NA",
           show_colnames = TRUE,
           color = viridis(n = 100),#colorRampPalette(rev(brewer.pal(6,"RdYlBu")))(50),#viridis(n = 100),
           cluster_cols = F,
           cluster_rows = F,
           annotation_col=col_anno,
           annotation_colors=col_list,
            legend=T,
            gaps_col=gap_col,
            annotation_row=anno_row,
            
           #scale="row",
        
           #clustering_distance_rows = "euclidean"#,
           #clustering_distance_cols = "euclidean"
            )
save_pheatmap_pdf(p,"WT.SCT.binary.regulon.res02_time-domain.rss_top20.7csimodule_WM.time.cluster.heatmap.viridis_2.pdf")

#### cis module average activity in spatial -220701

In [ ]:
meta<-read.csv("/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/res02_220310/WT.SCT.pc20.k50.res02.meta.data.csv")
rownames(meta)<-meta[,1]
meta<-meta[,-1]
head(meta)

In [ ]:
clusters_df<-read.csv("rss.top20.regulons.csi.7hcluster.df.csv")
head(clusters_df)

In [ ]:
colnames(auc_mtx)<-gsub("...$","",colnames(auc_mtx))
auc_mtx[1:5,1:5]

In [10]:
meta<-cbind(meta,auc_mtx)

In [ ]:
temp<-auc_mtx[,clusters_df$regulon]
dim(temp)

In [12]:
temp<-as.data.frame(t(temp))
temp$module<-clusters_df$csi_cluster

In [13]:
temp_m<-temp
temp_m$module<-as.character(temp_m$module)
temp_m<-aggregate(temp_m[,colnames(temp_m)!="module"],list(temp_m$module),mean)
head(temp_m)

In [ ]:
rownames(temp_m)<-temp_m[,1]
temp_m<-temp_m[,-1]
head(temp_m)

In [25]:
write.csv(temp_m,"7csi.module.average.auc.activity.eachSpot.df.csv")

In [ ]:
rownames(temp_m)<-paste0("module",rownames(temp_m))
head(temp_m)

In [ ]:
meta_m<-cbind(meta,t(temp_m))
dim(meta_m)

In [29]:
saveRDS(meta_m,"7csi.module.average.auc.activity.eachSpot.meta.rds")

In [15]:
meta_m<-readRDS("7csi.module.average.auc.activity.eachSpot.meta.rds")

In [3]:
meta_m<-readRDS("7csi.module.average.auc.activity.eachSpot.meta.rds")

In [6]:
sf<-"module.spatial_220720/"
if(!dir.exists(sf))
    dir.create(sf)

In [8]:
for(i in paste0("module",seq(1:7))){
    options(repr.plot.width=20,repr.plot.height=16)
    ggplot(meta_m,aes_string("x.ad","y.ad",color=i))+geom_point(size=1)+
        scale_color_gradientn(colours = c('#0000ff','#3232ff','#6666ff','#9898ff','#ccccff',
                                              '#fffefe','#ffcccc','#ff9898','#ff6666','#ff3232','#ff0000'))+
        #scale_fill_gradientn(colours = colorRampPalette(viridis(n = 10,option = "B"))(50))+
        xlab(paste0("")) +
        ylab(paste0("")) + 
        theme(panel.background = element_blank(),
              panel.grid.major =element_blank(), 
              panel.grid.minor = element_blank(),
              axis.text.x = element_blank(),#element_text(size = 15,colour = "black"),
              axis.text.y = element_blank()#element_text(size = 20,colour = "black")
              # axis.ticks.x = element_blank(),
        )
    ggsave(paste0(sf,"WT.merge.regress_CC.nC.mt.ident_",i,".spatial_manually_220720.png"),width=20,height=16,dpi=300)
}


In [ ]:
options(repr.plot.width=20,repr.plot.height=16)
ggplot(meta_m,aes_string("x.ad","y.ad",color="module1"))+geom_point(size=1)+
        #scale_colour_gradientn(colours=colorRampPalette(rev(brewer.pal(n = 11, name = "Spectral")))(50))+
        ggplot2::scale_colour_gradientn(colours = c('#0000ff','#3232ff','#6666ff','#9898ff','#ccccff',
                                              '#fffefe','#ffcccc','#ff9898','#ff6666','#ff3232','#ff0000'))+
        xlab(paste0("")) +
        ylab(paste0("")) + 
        theme(panel.background = element_blank(),
              panel.grid.major =element_blank(), 
              panel.grid.minor = element_blank(),
              axis.text.x = element_blank(),
              axis.text.y = element_blank()
          # axis.ticks.x = element_blank(),
    )#+